In [1]:
import pandas as pd

In [2]:
# read file
df = pd.read_table("MovieReview-Sample.csv",sep=',',header = None, names = ['text_no','review_text','labeled_sentiment'])
df.head()

,text_no,review_text,labeled_sentiment
0,1,films adapted from comic books have had plenty...,1
1,2,every now and then a movie comes along from a ...,1
2,3,you've got mail works alot better than it dese...,1
3,4,jaws is a rare film that grabs your attent...,1
4,5,moviemaking is a lot like being the general ma...,1


In [3]:
data = df.review_text.str.lower()

In [4]:
#Dictionary Approach
def count_pos_neg(data, positive_dict, negative_dict):
# count of positive and negative words that appeared in each message
# net count which is calculated by positive count subtracting negative count. 
    poscnt = []
    negcnt = []
    netcnt = []

    for nrow in range(0,len(data)):
        text = data[nrow]
        
        qa = 0
        qb = 0

        for word in positive_dict :
            if (word in text) :
                qa = qa + 1

        for word in negative_dict :
            if (word in text) :
                qb = qb + 1

        qc = qa - qb

        poscnt.append(qa)
        negcnt.append(qb)
        netcnt.append(qc)

    return (poscnt, negcnt, netcnt)

In [5]:
#Bing Liu's Dictionary
import nltk
nltk.download("opinion_lexicon")

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [6]:
#import Bing Liu's dictionary
from nltk.corpus import opinion_lexicon

In [7]:
pos_list_BL=set(opinion_lexicon.positive())
neg_list_BL=set(opinion_lexicon.negative())

In [8]:
df['poscnt_BL'], df['negcnt_BL'], df['netcnt_BL'] = count_pos_neg(data, pos_list_BL, neg_list_BL)

In [9]:
#show the df in BL
df[['review_text','poscnt_BL','negcnt_BL','netcnt_BL','labeled_sentiment']].head(5)

,review_text,poscnt_BL,negcnt_BL,netcnt_BL,labeled_sentiment
0,films adapted from comic books have had plenty...,16,36,-20,1
1,every now and then a movie comes along from a ...,21,33,-12,1
2,you've got mail works alot better than it dese...,32,28,4,1
3,jaws is a rare film that grabs your attent...,20,41,-21,1
4,moviemaking is a lot like being the general ma...,17,24,-7,1


In [10]:
#LM Dictionary
def read_local_dictionary(file):
    # create dictionary list
    words_dict = []
    with open(file, "r") as f: 
        for line in f:
            t = line.strip().lower()
            words_dict.append(t)
    return words_dict

In [11]:
pos_list_LM = read_local_dictionary('positive-words-LM.txt')
neg_list_LM = read_local_dictionary('negative-words-LM.txt')

In [12]:
df['poscnt_LM'], df['negcnt_LM'], df['netcnt_LM'] = count_pos_neg(data, pos_list_LM, neg_list_LM)

In [13]:
#show the df in LM
df[['review_text','poscnt_LM','negcnt_LM','netcnt_LM']].head(5)

,review_text,poscnt_LM,negcnt_LM,netcnt_LM
0,films adapted from comic books have had plenty...,5,11,-6
1,every now and then a movie comes along from a ...,7,12,-5
2,you've got mail works alot better than it dese...,12,7,5
3,jaws is a rare film that grabs your attent...,5,14,-9
4,moviemaking is a lot like being the general ma...,1,6,-5


In [14]:
#TextBlob
#!pip install textblob

In [15]:
from textblob import TextBlob

In [16]:
df["score_TextBlob"] = df["review_text"].map(lambda x:TextBlob(x).sentiment.polarity)

In [17]:
#show the df in TB
df[["review_text","score_TextBlob"]].head(5)

,review_text,score_TextBlob
0,films adapted from comic books have had plenty...,-0.061036
1,every now and then a movie comes along from a ...,0.088390
2,you've got mail works alot better than it dese...,0.081941
3,jaws is a rare film that grabs your attent...,0.066679
4,moviemaking is a lot like being the general ma...,0.054987


In [18]:
#Vader
#!pip install vadersentiment

In [19]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [20]:
analyzer = SentimentIntensityAnalyzer()
scores = [analyzer.polarity_scores(sentence) for sentence in data]

In [21]:
neg_s = [i["neg"] for i in scores]
neu_s = [i["neu"] for i in scores]
pos_s = [i["pos"] for i in scores]
compound_s = [i["compound"] for i in scores]

In [22]:
df['negscore_Vader'], df['neuscore_Vader'], df['posscore_Vader'], df['compound_Vader'] = neg_s, neu_s, pos_s, compound_s

In [23]:
#show the df in vader
df[['review_text','negscore_Vader','neuscore_Vader','posscore_Vader','compound_Vader']].head(5)

,review_text,negscore_Vader,neuscore_Vader,posscore_Vader,compound_Vader
0,films adapted from comic books have had plenty...,0.138,0.802,0.060,-0.9905
1,every now and then a movie comes along from a ...,0.069,0.833,0.098,0.8319
2,you've got mail works alot better than it dese...,0.075,0.765,0.160,0.9887
3,jaws is a rare film that grabs your attent...,0.085,0.806,0.109,0.9373
4,moviemaking is a lot like being the general ma...,0.037,0.849,0.114,0.9819


In [24]:
#show the df in all
df.head()

,text_no,review_text,labeled_sentiment,poscnt_BL,negcnt_BL,netcnt_BL,poscnt_LM,negcnt_LM,netcnt_LM,score_TextBlob,negscore_Vader,neuscore_Vader,posscore_Vader,compound_Vader
0,1,films adapted from comic books have had plenty...,1,16,36,-20,5,11,-6,-0.061036,0.138,0.802,0.060,-0.9905
1,2,every now and then a movie comes along from a ...,1,21,33,-12,7,12,-5,0.088390,0.069,0.833,0.098,0.8319
2,3,you've got mail works alot better than it dese...,1,32,28,4,12,7,5,0.081941,0.075,0.765,0.160,0.9887
3,4,jaws is a rare film that grabs your attent...,1,20,41,-21,5,14,-9,0.066679,0.085,0.806,0.109,0.9373
4,5,moviemaking is a lot like being the general ma...,1,17,24,-7,1,6,-5,0.054987,0.037,0.849,0.114,0.9819


In [25]:
#show df1 only contain 'labeled_sentiment', 'review_text','netcnt_BL', 'netcnt_LM', 'score_TextBlob', 'compound_Vader'
df1 = df[['labeled_sentiment', 'review_text','netcnt_BL', 'netcnt_LM', 'score_TextBlob', 'compound_Vader']]
df1.head()

,labeled_sentiment,review_text,netcnt_BL,netcnt_LM,score_TextBlob,compound_Vader
0,1,films adapted from comic books have had plenty...,-20,-6,-0.061036,-0.9905
1,1,every now and then a movie comes along from a ...,-12,-5,0.088390,0.8319
2,1,you've got mail works alot better than it dese...,4,5,0.081941,0.9887
3,1,jaws is a rare film that grabs your attent...,-21,-9,0.066679,0.9373
4,1,moviemaking is a lot like being the general ma...,-7,-5,0.054987,0.9819


In [26]:
#make copy
df2 = df1.copy()
# get the Bing Liu Lexicon, if netcnt_BL > 0, the BL_score is 1, if not, the BL_score is 0
df2.loc[df1['netcnt_BL'] > 0, 'BL_score'] = 1
df2.loc[df1['netcnt_BL'] <= 0, 'BL_score'] = 0

# get the LM dictionary, if netcnt_LM > 0, the LM_score is 1, if not, the LM_score is 0
df2.loc[df1['netcnt_LM'] > 0, 'LM_score'] = 1
df2.loc[df1['netcnt_LM'] <= 0, 'LM_score'] = 0

# get the TextBlob, if score_TextBlob > 0, the TB_score is 1, if not, the TB_score is 0
df2.loc[df1['score_TextBlob'] > 0, 'TB_score'] = 1
df2.loc[df1['score_TextBlob'] <= 0, 'TB_score'] = 0

# get the Vader score, if compound_Vader > 0, the VD_score is 1, if not, the VD_score is 0
df2.loc[df1['compound_Vader'] > 0, 'VD_score'] = 1
df2.loc[df1['compound_Vader'] <= 0, 'VD_score'] = 0

In [27]:
#show df2
df2.head()

,labeled_sentiment,review_text,netcnt_BL,netcnt_LM,score_TextBlob,compound_Vader,BL_score,LM_score,TB_score,VD_score
0,1,films adapted from comic books have had plenty...,-20,-6,-0.061036,-0.9905,0.0,0.0,0.0,0.0
1,1,every now and then a movie comes along from a ...,-12,-5,0.088390,0.8319,0.0,0.0,1.0,1.0
2,1,you've got mail works alot better than it dese...,4,5,0.081941,0.9887,1.0,1.0,1.0,1.0
3,1,jaws is a rare film that grabs your attent...,-21,-9,0.066679,0.9373,0.0,0.0,1.0,1.0
4,1,moviemaking is a lot like being the general ma...,-7,-5,0.054987,0.9819,0.0,0.0,1.0,1.0


In [28]:
# make df3 only have 'labeled_sentiment', 'BL_score', 'LM_score', 'TB_score', 'VD_score' column
df3 = df2[['labeled_sentiment', 'BL_score', 'LM_score', 'TB_score', 'VD_score']]
df3.head()

,labeled_sentiment,BL_score,LM_score,TB_score,VD_score
0,1,0.0,0.0,0.0,0.0
1,1,0.0,0.0,1.0,1.0
2,1,1.0,1.0,1.0,1.0
3,1,0.0,0.0,1.0,1.0
4,1,0.0,0.0,1.0,1.0


In [29]:
from sklearn import metrics

In [30]:
# create confusion matrix for BL_score
matrixBL = metrics.confusion_matrix(df3['labeled_sentiment'], df3['BL_score'])
matrixBL

array([[966,  34],
       [867, 133]], dtype=int64)

In [31]:
# create confusion matrix for LM_score 
matrixLM = metrics.confusion_matrix(df3['labeled_sentiment'], df3['LM_score'])
matrixLM

array([[931,  69],
       [840, 160]], dtype=int64)

In [32]:
# create confusion matrix for TB_score 
matrixTB = metrics.confusion_matrix(df3['labeled_sentiment'], df3['TB_score'])
matrixTB

array([[274, 726],
       [ 79, 921]], dtype=int64)

In [33]:
# create confusion matrix for VD_score 
matrixV = metrics.confusion_matrix(df3['labeled_sentiment'], df3['VD_score'])
matrixV

array([[489, 511],
       [268, 732]], dtype=int64)

In [34]:
# define the function to calculation of precision, recall and F_measure
def validation(y_actual, y_predicted):
    precision = round(metrics.precision_score(y_actual, y_predicted), 3)
    recall = round(metrics.recall_score(y_actual, y_predicted), 3)
    F_measure = round((2 * precision * recall) / (precision + recall), 3)
    return precision, recall, F_measure

In [35]:
# get the precision, recall and F_measure of each approach
matrixBL = validation(df3['labeled_sentiment'], df3['BL_score'])
matrixLM = validation(df3['labeled_sentiment'], df3['LM_score'])
matrixTB = validation(df3['labeled_sentiment'], df3['TB_score'])
matrixV = validation(df3['labeled_sentiment'], df3['VD_score'])

In [36]:
#show each approaches' 'precision', 'recall', 'F_measure'
result = pd.DataFrame([matrixBL, matrixLM, matrixTB, matrixV], columns = ['precision', 'recall', 'F_measure'] )
result.insert(0, 'approach',['BL', 'LM', 'TB', 'Vader'], True)
result

,approach,precision,recall,F_measure
0,BL,0.796,0.133,0.228
1,LM,0.699,0.160,0.260
2,TB,0.559,0.921,0.696
3,Vader,0.589,0.732,0.653


In [37]:
# Highlighting the maximum values of each column
result.style.highlight_max(color = 'red', axis = 0)

,approach,precision,recall,F_measure
0,BL,0.796000,0.133000,0.228000
1,LM,0.699000,0.160000,0.260000
2,TB,0.559000,0.921000,0.696000
3,Vader,0.589000,0.732000,0.653000


# Q2

In [38]:
#!pip install numpy

In [39]:
import numpy
numpy.__version__

'1.20.1'

In [40]:
import matplotlib.pyplot as plt
import re
import os
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics



In [41]:
# Add 3 scores get majority vote
df3['vote'] =  df3['BL_score'] + df3['TB_score'] + df3['VD_score']
df3.head()

<ipython-input-41-dc60177ae04a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['vote'] =  df3['BL_score'] + df3['TB_score'] + df3['VD_score']


,labeled_sentiment,BL_score,LM_score,TB_score,VD_score,vote
0,1,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,1.0,1.0,2.0
2,1,1.0,1.0,1.0,1.0,3.0
3,1,0.0,0.0,1.0,1.0,2.0
4,1,0.0,0.0,1.0,1.0,2.0


In [42]:
# Use majority vote to get new vote , if new vote > 1, new vote = 1, else new vote = 0
df3.loc[df3['vote'] > 1, 'score_vote'] = 1
df3.loc[df3['vote'] <= 1, 'score_vote'] = 0
df3.head()

D:\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,labeled_sentiment,BL_score,LM_score,TB_score,VD_score,vote,score_vote
0,1,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,1.0,1.0,2.0,1.0
2,1,1.0,1.0,1.0,1.0,3.0,1.0
3,1,0.0,0.0,1.0,1.0,2.0,1.0
4,1,0.0,0.0,1.0,1.0,2.0,1.0


In [43]:
# use defined the validation function
vote_validation = validation(df3['labeled_sentiment'], df3['score_vote'])
vote_validation

(0.613, 0.715, 0.66)

In [44]:
result2 = pd.DataFrame([matrixBL, matrixTB, matrixV, vote_validation], columns = [ 'precision', 'recall', 'F_measure'] )
result2

,precision,recall,F_measure
0,0.796,0.133,0.228
1,0.559,0.921,0.696
2,0.589,0.732,0.653
3,0.613,0.715,0.660


In [45]:
#add the name to the result2
result2.insert(0, 'approach', ['BingLiu Lexicon', 'TextBlog', 'Vader', 'Major_vote'], True)
result2

,approach,precision,recall,F_measure
0,BingLiu Lexicon,0.796,0.133,0.228
1,TextBlog,0.559,0.921,0.696
2,Vader,0.589,0.732,0.653
3,Major_vote,0.613,0.715,0.660


In [46]:
# calculate improvement percentage change
def improvement(new, old):
    pre_change=round((new[0]-old[0])/old[0],3)
    rec_change=round((new[1]-old[1])/old[1],3)
    F_change=round((new[2]-old[2])/old[2],3)
    return pre_change, rec_change, F_change

In [47]:
matrixBL, matrixTB, matrixV, vote_validation

((0.796, 0.133, 0.228),
 (0.559, 0.921, 0.696),
 (0.589, 0.732, 0.653),
 (0.613, 0.715, 0.66))

In [48]:
# vote 2 improvement compairng to vote 
improve_BL2 = improvement(vote_validation, matrixBL)
improve_TB2 = improvement(vote_validation, matrixTB)
improve_VD2 = improvement(vote_validation, matrixV)

In [49]:
# add the improvements to result1
result3 = pd.DataFrame([improve_BL2, improve_TB2, improve_VD2], columns = [ 'precision', 'recall', 'F_measure'] )

In [50]:
# add the vote to the result1 dataframe
result3.insert(0, 'approach', ['improve_BL2',  'improve_TB2', 'improve_VD2'], True)

In [51]:
# Combine 2 dataframe together
df_final = pd.concat([result2, result3], ignore_index=True, axis=0)
df_final

,approach,precision,recall,F_measure
0,BingLiu Lexicon,0.796,0.133,0.228
1,TextBlog,0.559,0.921,0.696
2,Vader,0.589,0.732,0.653
3,Major_vote,0.613,0.715,0.660
4,improve_BL2,-0.230,4.376,1.895
5,improve_TB2,0.097,-0.224,-0.052
6,improve_VD2,0.041,-0.023,0.011
